# Project 3: LangChain Function/Tool Calling Calculator
In this Project, we will create a simple LangChain Colab Notebook that uses the Google Gemini Flash model to answer user calculation questions. This example below is provided to help you get started assumes you have access to the Gemini API and a basic Python environment. However, you are required to develop and submit your project using Google Colab.

In [20]:
!pip install python-dotenv -q
!pip install langchain-community -U -q
!pip install google-generativeai -U -q # Update `google-generativeai` package
!pip install langchain -U -q
!pip install langchain_google_genai -U -q

### Step 1: Import API keys  in Colab from secrets


In [82]:
#from google.colab import userdata
#userdata.get('GOOGLE_API_KEY')

#### Step 2: Define the Calculator Tool
We define a Calculator class that can evaluate arithmetic expressions. The calculate method uses Python's eval function to compute the result safely.

In [72]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

#### Step 3: Create the Tool Wrapper for LangChain
We integrate the calculator with LangChain as a tool using the Tool class.

In [73]:
from langchain.tools import tool

# Define the tool using a decorator
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

#### Step 4: Set Up the Google Gemini Flash Model
We initialize the Google Gemini Flash model with the API key and define the tools to be used by the agent.

In [74]:
import os
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains.conversation.memory import ConversationBufferMemory

# Initialize the Google Gemini Flash model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model name, e.g., 'gemini-1.5-flash'
    api_key=os.environ["GOOGLE_GEMINI_API_KEY"],  # Use the environment variable for the API key
    temperature=0.7
)
print("Google Gemini Flash model initialized.")


Google Gemini Flash model initialized.


In [77]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool


# Create a list of tools
tools = [calculator]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=gemini_model,
    agent_type="openai-function-calling"
)

# Example usage
response = agent.invoke("What is 12 * 15?")
print(response)



{'input': 'What is 12 * 15?', 'output': '180'}


### Step 5: Build the Conversational Chain
We set up a conversational chain with memory to maintain context across multiple queries.


In [78]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Set up memory to maintain conversation context
memory = ConversationBufferMemory()

# Build the conversational chain
chain = ConversationChain(
    llm=gemini_model,
    memory=memory,
    verbose=True
)

# Example usage
response = chain.invoke("What is 12 * 15?")
print(response)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 12 * 15?
AI:

> Finished chain.
{'input': 'What is 12 * 15?', 'history': '', 'response': '12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I can break it down a few ways:  I could do 12 * 10 = 120, and then 12 * 5 = 60, and add those together (120 + 60 = 180). Or, I could use the distributive property: 12 * (10 + 5) = (12 * 10) + (12 * 5) = 120 + 60 = 180.  There are many ways to approach this simple multiplication problem!  Is there anything else I can help you calculate?\n'}


#### Step 6: Test the Calculator Tool
We test the calculator tool by processing a series of queries and printing the responses.

In [79]:
# User query
query = "What is 15 divided by 3?"

# Get response from the system
response = chain.invoke(query)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I can break it down a few ways:  I could do 12 * 10 = 120, and then 12 * 5 = 60, and add those together (120 + 60 = 180). Or, I could use the distributive property: 12 * (10 + 5) = (12 * 10) + (12 * 5) = 120 + 60 = 180.  There are many ways to approach this simple multiplication problem!  Is there anything else I can help you calculate?

Human: What is 15 divided by 3?
AI:

> Finished chain.
{'input': 'What is 15 divided by 3?', 'history': 'Human: What is 12 * 15?\nAI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication

In [81]:

queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.invoke(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at this answer by performing the multiplication operation.  Specifically, I can break it down a few ways:  I could do 12 * 10 = 120, and then 12 * 5 = 60, and add those together (120 + 60 = 180). Or, I could use the distributive property: 12 * (10 + 5) = (12 * 10) + (12 * 5) = 120 + 60 = 180.  There are many ways to approach this simple multiplication problem!  Is there anything else I can help you calculate?

Human: What is 15 divided by 3?
AI: 15 divided by 3 is 5.  This is because 3 multiplied by 5 equals 15.  In other words, 3 goes into 15 five times.  I performed this calcula

--------------------